In [23]:
import pandas as pd

In [2]:
customer = pd.read_csv('uselog_joined.csv')
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47
5,IK271057,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,3.750000,3.5,5,2,1,2019-04-30,47
6,HI324666,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,3.916667,3.5,7,2,1,2019-04-30,47
7,AS647421,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.500000,4.0,9,2,1,2019-04-30,47
8,AS069821,XXXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,4.083333,3.5,7,2,1,2019-04-30,47
9,GD039425,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.666667,4.5,7,3,1,2019-04-30,47


In [3]:
uselog_months = pd.read_csv('use_log_months.csv')
uselog_months

,ym,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
5,201804,AS015746,4
6,201804,AS019860,6
7,201804,AS024181,4
8,201804,AS030404,1
9,201804,AS031206,4


In [4]:
# 顧客の退会を予測する

In [5]:
# 当月と前月の利用履歴のみを利用する
yms = list(uselog_months["ym"].unique())
uselog = pd.DataFrame()

for i in range(1, len(yms)):
    # 当月
    tmp = uselog_months.loc[uselog_months["ym"]==yms[i]]
    tmp.rename(columns={"count":"count_t"}, inplace=True)

    # 前月
    tmp_before = uselog_months.loc[uselog_months["ym"]==yms[i-1]]
    del tmp_before["ym"]
    tmp_before.rename(columns={"count":"count_p"}, inplace=True)
    
    # customer_id で結合
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    
    # ユニオン
    uselog = pd.concat([uselog, tmp], ignore_index=True)

/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [6]:
uselog

,ym,customer_id,count_t,count_p
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
5,201805,AS015746,1,4.0
6,201805,AS019860,8,6.0
7,201805,AS024181,6,4.0
8,201805,AS025956,6,NaN
9,201805,AS029624,8,NaN


In [7]:
# 退会顧客の退会前月のデータを作成する
# 退会月の前月に退会申請を出すシステムなので、退会月の前月のデータを見て、翌月に退会するかどうかを予測するのがねらい

In [8]:
from dateutil.relativedelta import relativedelta

In [13]:
exit_customer = customer.loc[customer["is_deleted"]==1] # 退会済
exit_customer["exit_date"] = None # 退会申請月（＝退会月の前月）
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a co

In [15]:
exit_customer["ym"] = exit_customer["exit_date"].dt.strftime("%Y%m")

/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration,exit_date,ym
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.000000,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.000000,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.000000,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.500000,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.000000,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00,201804
813,OA007936,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.000000,3.0,4,2,0,2018-05-31,23,2018-04-30 00:00:00,201804
837,AS842977,XXXXX,C02,F,2016-06-01,2018-05-31,CA1,1,デイタイム,7500,通常,4.000000,4.0,5,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
846,TS915632,XXXXX,C02,F,2016-07-01,2018-04-30,CA2,1,デイタイム,7500,入会費半額,3.000000,3.0,3,3,0,2018-04-30,21,2018-03-30 00:00:00,201803
847,HD805772,XXXX,C02,F,2016-07-01,2018-05-31,CA2,1,デイタイム,7500,入会費半額,3.000000,3.0,3,3,0,2018-05-31,22,2018-04-30 00:00:00,201804
857,IK942599,XXXXX,C02,F,2016-07-01,2018-05-31,CA2,1,デイタイム,7500,入会費半額,1.500000,1.5,2,1,0,2018-05-31,22,2018-04-30 00:00:00,201804


In [17]:
uselog["ym"] = uselog["ym"].astype(str)

In [18]:
# 退会済の顧客の退会前月のデータと結合するので、それ以外の年月・顧客のデータは不要
# leftjoinして欠損値をdropnaしてもよいが、ここではinnerjoinで結合する
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "ym"], how="inner")

In [19]:
exit_uselog

,ym,customer_id,count_t,count_p,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration,exit_date
0,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500,通常,3.000000,3.0,3,3,0,2018-06-30,3,2018-05-30 00:00:00
1,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500,通常,3.000000,3.0,4,2,1,2018-06-30,7,2018-05-30 00:00:00
2,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500,通常,3.333333,3.0,5,2,0,2018-06-30,12,2018-05-30 00:00:00
3,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500,通常,3.333333,3.0,5,2,0,2018-06-30,4,2018-05-30 00:00:00
4,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000,通常,2.000000,1.0,4,1,0,2018-06-30,9,2018-05-30 00:00:00
5,201805,AS450151,1,9.0,XXXXX,C02,F,2018-04-03,2018-06-30,CA3,...,7500,入会費無料,4.333333,3.0,9,1,0,2018-06-30,2,2018-05-30 00:00:00
6,201805,AS525250,9,NaN,XXXX,C03,F,2018-05-09,2018-06-30,CA2,...,6000,入会費半額,5.500000,5.5,9,2,1,2018-06-30,1,2018-05-30 00:00:00
7,201805,AS543483,4,1.0,XXXXX,C03,M,2017-09-01,2018-06-30,CA1,...,6000,通常,2.666667,3.0,4,1,0,2018-06-30,9,2018-05-30 00:00:00
8,201805,AS723178,4,1.0,XX,C01,M,2017-07-01,2018-06-30,CA2,...,10500,入会費半額,2.333333,2.0,4,1,0,2018-06-30,11,2018-05-30 00:00:00
9,201805,AS824458,3,5.0,XXXXX,C03,M,2017-07-01,2018-06-30,CA2,...,6000,入会費半額,3.333333,3.0,5,2,1,2018-06-30,11,2018-05-30 00:00:00


In [20]:
# 件数の確認 退会済顧客ごとに1件（退会前月のみ）のデータになっているはず
len(exit_uselog["customer_id"].unique())

1104

In [39]:
# 継続顧客のデータ
cont_customer = customer.loc[customer["is_deleted"]==0] # 継続中
cont_uselog = pd.merge(uselog, cont_customer, on=["customer_id"], how="inner")

In [40]:
cont_uselog

,ym,customer_id,count_t,count_p,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
1,201806,AS002855,5,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
2,201807,AS002855,5,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
3,201808,AS002855,3,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
4,201809,AS002855,7,3.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
5,201810,AS002855,3,7.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
6,201811,AS002855,5,3.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
7,201812,AS002855,2,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
8,201901,AS002855,6,2.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
9,201902,AS002855,5,6.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29


In [43]:
# データ件数が退会：継続＝1,104:27,422と不均衡なので、アンダーサンプリングする
cont_uselog = cont_uselog.sample(frac=1).reset_index(drop=True) # 元のデータをランダムに並び替え
cont_uselog = cont_uselog.drop_duplicates(subset="customer_id") # customer_id ごとに先頭1行のみを残し、それ以外を削除

In [44]:
cont_uselog

,ym,customer_id,count_t,count_p,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,end_date_c,duration
0,201901,TS806267,7,5.0,XXX,C03,M,2017-09-01,NaN,CA1,...,ナイト,6000,通常,6.250000,6.0,9,3,1,2019-04-30,19
1,201812,OA500718,4,7.0,XXXXX,C01,M,2016-10-01,NaN,CA1,...,オールタイム,10500,通常,5.083333,5.5,8,1,1,2019-04-30,30
2,201903,PL852196,7,8.0,XXXXXX,C01,M,2019-01-11,NaN,CA1,...,オールタイム,10500,通常,8.333333,8.0,10,7,1,2019-04-30,3
3,201805,IK046372,5,7.0,XX,C01,F,2016-11-01,NaN,CA1,...,オールタイム,10500,通常,4.750000,5.0,7,3,1,2019-04-30,29
4,201805,AS992593,6,8.0,XXXX,C03,F,2017-04-01,NaN,CA1,...,ナイト,6000,通常,5.583333,5.0,8,4,1,2019-04-30,24
5,201808,HI755073,5,4.0,XXX,C03,M,2017-05-01,NaN,CA1,...,ナイト,6000,通常,5.666667,6.0,7,4,1,2019-04-30,23
6,201808,IK797502,7,6.0,XXXX,C03,M,2018-01-01,NaN,CA1,...,ナイト,6000,通常,6.583333,7.0,9,4,1,2019-04-30,15
7,201812,HI604733,6,1.0,XXXXX,C03,M,2016-06-01,NaN,CA1,...,ナイト,6000,通常,4.250000,4.5,7,1,1,2019-04-30,34
8,201806,TS784450,7,8.0,XXX,C03,F,2017-10-01,NaN,CA1,...,ナイト,6000,通常,6.583333,6.5,9,4,1,2019-04-30,18
9,201810,OA025439,2,8.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,オールタイム,10500,通常,4.583333,4.0,8,2,1,2019-04-30,45


In [45]:
#  退会・継続のデータをユニオン
predict_data = pd.concat([cont_uselog, exit_uselog], ignore_index=True)

In [46]:
predict_data

,campaign_id,campaign_name,class,class_name,count_p,count_t,customer_id,duration,end_date,end_date_c,...,is_deleted,max,mean,median,min,name,price,routine_flg,start_date,ym
0,CA1,通常,C03,ナイト,5.0,7,TS806267,19,NaN,2019-04-30,...,0,9,6.250000,6.0,3,XXX,6000,1,2017-09-01,201901
1,CA1,通常,C01,オールタイム,7.0,4,OA500718,30,NaN,2019-04-30,...,0,8,5.083333,5.5,1,XXXXX,10500,1,2016-10-01,201812
2,CA1,通常,C01,オールタイム,8.0,7,PL852196,3,NaN,2019-04-30,...,0,10,8.333333,8.0,7,XXXXXX,10500,1,2019-01-11,201903
3,CA1,通常,C01,オールタイム,7.0,5,IK046372,29,NaN,2019-04-30,...,0,7,4.750000,5.0,3,XX,10500,1,2016-11-01,201805
4,CA1,通常,C03,ナイト,8.0,6,AS992593,24,NaN,2019-04-30,...,0,8,5.583333,5.0,4,XXXX,6000,1,2017-04-01,201805
5,CA1,通常,C03,ナイト,4.0,5,HI755073,23,NaN,2019-04-30,...,0,7,5.666667,6.0,4,XXX,6000,1,2017-05-01,201808
6,CA1,通常,C03,ナイト,6.0,7,IK797502,15,NaN,2019-04-30,...,0,9,6.583333,7.0,4,XXXX,6000,1,2018-01-01,201808
7,CA1,通常,C03,ナイト,1.0,6,HI604733,34,NaN,2019-04-30,...,0,7,4.250000,4.5,1,XXXXX,6000,1,2016-06-01,201812
8,CA1,通常,C03,ナイト,8.0,7,TS784450,18,NaN,2019-04-30,...,0,9,6.583333,6.5,4,XXX,6000,1,2017-10-01,201806
9,CA1,通常,C01,オールタイム,8.0,2,OA025439,45,NaN,2019-04-30,...,0,8,4.583333,4.0,2,XXXXX,10500,1,2015-07-01,201810


In [64]:
# 変数として在籍期間のカラムを追加
predict_data["period"] = None
predict_data["now_date"] = pd.to_datetime(predict_data["ym"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

In [67]:
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = delta.years * 12 + delta.months

/Users/y-uchiyama/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
predict_data.head(1000)

,campaign_id,campaign_name,class,class_name,count_p,count_t,customer_id,duration,end_date,end_date_c,...,mean,median,min,name,price,routine_flg,start_date,ym,period,now_date
0,CA1,通常,C03,ナイト,5.0,7,TS806267,19,NaN,2019-04-30,...,6.250000,6.0,3,XXX,6000,1,2017-09-01,201901,16,2019-01-01
1,CA1,通常,C01,オールタイム,7.0,4,OA500718,30,NaN,2019-04-30,...,5.083333,5.5,1,XXXXX,10500,1,2016-10-01,201812,26,2018-12-01
2,CA1,通常,C01,オールタイム,8.0,7,PL852196,3,NaN,2019-04-30,...,8.333333,8.0,7,XXXXXX,10500,1,2019-01-11,201903,1,2019-03-01
3,CA1,通常,C01,オールタイム,7.0,5,IK046372,29,NaN,2019-04-30,...,4.750000,5.0,3,XX,10500,1,2016-11-01,201805,18,2018-05-01
4,CA1,通常,C03,ナイト,8.0,6,AS992593,24,NaN,2019-04-30,...,5.583333,5.0,4,XXXX,6000,1,2017-04-01,201805,13,2018-05-01
5,CA1,通常,C03,ナイト,4.0,5,HI755073,23,NaN,2019-04-30,...,5.666667,6.0,4,XXX,6000,1,2017-05-01,201808,15,2018-08-01
6,CA1,通常,C03,ナイト,6.0,7,IK797502,15,NaN,2019-04-30,...,6.583333,7.0,4,XXXX,6000,1,2018-01-01,201808,7,2018-08-01
7,CA1,通常,C03,ナイト,1.0,6,HI604733,34,NaN,2019-04-30,...,4.250000,4.5,1,XXXXX,6000,1,2016-06-01,201812,30,2018-12-01
8,CA1,通常,C03,ナイト,8.0,7,TS784450,18,NaN,2019-04-30,...,6.583333,6.5,4,XXX,6000,1,2017-10-01,201806,8,2018-06-01
9,CA1,通常,C01,オールタイム,8.0,2,OA025439,45,NaN,2019-04-30,...,4.583333,4.0,2,XXXXX,10500,1,2015-07-01,201810,39,2018-10-01


In [70]:
# 欠損値の除去
predict_data.isnull().sum()
# 前月利用回数がないデータ(count_pがNaN)は使えないので、これだけ除去する

campaign_id         0
campaign_name       0
class               0
class_name          0
count_p           247
count_t             0
customer_id         0
duration            0
end_date         2842
end_date_c          0
exit_date        2842
gender              0
is_deleted          0
max                 0
mean                0
median              0
min                 0
name                0
price               0
routine_flg         0
start_date          0
ym                  0
period              0
now_date            0
dtype: int64

In [76]:
predict_data = predict_data.dropna(subset=["count_p"])
predict_data.isnull().sum()

campaign_id         0
campaign_name       0
class               0
class_name          0
count_p             0
count_t             0
customer_id         0
duration            0
end_date         2647
end_date_c          0
exit_date        2647
gender              0
is_deleted          0
max                 0
mean                0
median              0
min                 0
name                0
price               0
routine_flg         0
start_date          0
ym                  0
period              0
now_date            0
dtype: int64

In [80]:
predict_save = predict_data

In [89]:
# 区分、性別などの文字列データ(カテゴリカル変数)を活用するために、ダミー変数化(フラグ化)する
predict_data = predict_data[["campaign_name", "class_name", "gender", "count_p", "routine_flg", "period", "is_deleted"]]
predict_data

,campaign_name,class_name,gender,count_p,routine_flg,period,is_deleted
0,通常,ナイト,M,5.0,1,16,0
1,通常,オールタイム,M,7.0,1,26,0
2,通常,オールタイム,M,8.0,1,1,0
3,通常,オールタイム,F,7.0,1,18,0
4,通常,ナイト,F,8.0,1,13,0
5,通常,ナイト,M,4.0,1,15,0
6,通常,ナイト,M,6.0,1,7,0
7,通常,ナイト,M,1.0,1,30,0
8,通常,ナイト,F,8.0,1,8,0
9,通常,オールタイム,M,8.0,1,39,0


In [90]:
predict_data["period"] = predict_data["period"].astype(int)

In [91]:
predict_data.dtypes

campaign_name     object
class_name        object
gender            object
count_p          float64
routine_flg        int64
period             int64
is_deleted         int64
dtype: object

In [92]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_p,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1,16,0,0,0,1,0,0,1,0,1
1,7.0,1,26,0,0,0,1,1,0,0,0,1
2,8.0,1,1,0,0,0,1,1,0,0,0,1
3,7.0,1,18,0,0,0,1,1,0,0,1,0
4,8.0,1,13,0,0,0,1,0,0,1,1,0


In [93]:
# 区分ごとに、バリエーションのうち一つは「どれにもフラグが立っていない」ことで表現できるため、
# 1つずつは削除する必要がある
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_p,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1,16,0,0,0,0,0,0
1,7.0,1,26,0,0,0,1,0,0
2,8.0,1,1,0,0,0,1,0,0
3,7.0,1,18,0,0,0,1,0,1
4,8.0,1,13,0,0,0,0,0,1


In [95]:
# 決定木を用いて、退会予測モデルを作成する
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [97]:
exit = predict_data.loc[predict_data["is_deleted"]==1]
cont = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit)) # nの指定：退会と継続で件数をあわせる

In [98]:
# 説明変数X：退会フラグを除く全ての変数
# 目的変数ｙ：退会フラグ
X = pd.concat([exit, cont], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [106]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train) # 説明変数・目的変数の教師データでモデルを構築
y_test_pred = model.predict(X_test) # 構築したモデルに説明変数の評価データを与えて、予測された目的変数を結果として得る
y_test_pred
# 1は退会、0は継続と予測されたことを意味する

array([1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1,

In [107]:
# 予測結果と、評価データとの比較
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head(20)

,y_pred,y_test
552,1,1
308,1,1
1179,0,0
1496,0,0
949,1,1
245,1,1
2087,0,0
2057,0,0
2015,0,0
1125,0,0


In [109]:
# 予測モデルの評価
# 正解数 / 全データ件数　で　正解率を計算
correct = len(results_test.loc[results_test["y_pred"]==results_test["y_test"]])
data_count = len(results_test)
score_test = correct / data_count
score_test

0.8992395437262357

In [110]:
# 90%近い精度であることがわかったが・・・
# 学習用データでの予測精度と、評価用データでの予測精度の差が小さいことが理想的
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.899239543726
0.982256020279


In [111]:
# 学習用データの精度は98%で、過学習傾向であると言える　→　チューニングが必要

In [112]:
# そこで、データの件数や使う変数、モデルのパラメータを見直してみる

In [113]:
# 決定木の深さを調節して、再度モデルを構築
X = pd.concat([exit, cont], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.904942965779
0.927122940431


In [114]:
# テストデータ90%, 学習データ92%
# 未知のデータにも対応できるそこそこのモデルができたのではないか

In [115]:
# モデルに寄与している変数を確認する
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})

In [117]:
importance
# 会員期間と前月の利用回数が大きく影響、定期利用しているかどうかもそれなりに影響

,coefficient,feature_names
0,0.358574,count_p
1,0.137922,routine_flg
2,0.503504,period
3,0.000000,campaign_name_入会費半額
4,0.000000,campaign_name_入会費無料
5,0.000000,class_name_オールタイム
6,0.000000,class_name_デイタイム
7,0.000000,gender_F


In [119]:
# 決定木構造の可視化（dotファイルを生成）
sklearn.tree.export_graphviz(model, out_file='tree.dot')

In [121]:
# dotファイルをpng画像にする
# →ターミナルでコマンド実行　dot -Tpng tree.dot -o tree.png

In [122]:
# 顧客の退会を予測する

In [145]:
# モデル作成時に使用した説明変数を引き渡して、退会確率を予測してみる
# いろいろ入力を変えて、予測結果で遊んでみよう
count_p = 10
routine_flg = 1
period = 10
campaign_name_list = [0, 1] # 入会費半額、入会費無料、通常
class_name_list = [1, 0] # オールタイム、デイタイム、ナイト
gender = [0] # 1:F, 0:M

In [146]:
input_data = [count_p, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender)

In [147]:
# 予測結果
print(model.predict([input_data]))
# 退会するかどうかの確率
print(model.predict_proba([input_data]))

[0]
[[ 0.93956044  0.06043956]]


In [138]:
# モデルが構築できたら、これをシステムに組み込んで、随時インプットを与えて予測結果を返すようなツールを作れると良い。